In [1]:
from google.colab import drive
import json
import os
import pandas as pd
import numpy as np
import random
import tensorflow as tf
import torch
from sklearn.metrics import accuracy_score, roc_auc_score, average_precision_score, matthews_corrcoef, f1_score
from keras.models import Sequential, Model
from keras.layers import Conv2D, Conv3D, BatchNormalization, Activation, Dense, Flatten, Dropout, Input, concatenate, GlobalAveragePooling2D, GlobalAveragePooling3D, TimeDistributed, LSTM
from keras.losses import mean_squared_error, huber, log_cosh
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau, TensorBoard
from keras.regularizers import l1_l2
from keras import backend as K
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight
from sklearn.model_selection import GridSearchCV, StratifiedKFold
import datetime
import h5py
from tensorflow.python.keras.saving import hdf5_format
import seaborn as sns
import pickle

In [2]:
# Mount drive
drive.mount('/content/drive', force_remount=True)

# Set root path
root_path = "drive/MyDrive/NNextPitch"

# Open data
with open(os.path.join(root_path, "data/modeling-data/modeling-data.glasnow.json")) as f:
    data = json.load(f)

Mounted at /content/drive


# Modeling Preparation

In [3]:
def extract_xy(data_dict: dict, outcome_key: str):

  x = []
  y = []

  for play_id in data_dict.keys():

    # Skip NAs
    if type(data_dict[play_id]["outcome2"]) != str:
      print(f"{play_id} has NA outcome. Skipping sample.\n")

    else:
      # Get play-specific pose data
      pose_dict = data_dict[play_id]['pose_data']

      # Re-organize play-specific data into shape (10, 15, 2)
      # Shape: 
      # [[x0, y0], [x1, y1], ..., [x15, y15],
      #  [x0, y0], [x1, y1], ..., [x15, y15],
      #  ... 10dims, 1 per keypoint ...
      #  [x0, y0], [x1, y1], ..., [x15, y15]]
      play_output = []
      for keypoint in pose_dict.keys():
        xy_dict = pose_dict[keypoint]
        play_output.append(np.array([np.array(xy) for xy in zip(xy_dict['x'], xy_dict['y'])]))
      play_output = np.array(play_output)

      # Add re-organized play-specific data to the full output set
      x.append(play_output)

      # Extract y
      y.append(data_dict[play_id][outcome_key])

  # Encode y
  outcomes = list(np.unique(y))
  outcome_map = {pitch_type: i for i, pitch_type in enumerate(outcomes)}
  y = pd.Series(y).map(outcome_map)
  print(f"Outcome encoding:\n{outcome_map}")

  # Return x, y
  return np.array(x), np.array(y)


In [4]:
x, y = extract_xy(data, outcome_key="outcome1")

8182ea7a-f152-4fb3-af39-5d468190ddbe has NA outcome. Skipping sample.

Outcome encoding:
{'FB': 0, 'OFF': 1}


In [5]:
# Check shapes of x and y
print(x.shape)
print(y.shape)

(719, 10, 15, 2)
(719,)


In [6]:
# Reshape to make time the third dimension
x = np.transpose(x, (0, 2, 1, 3))
print(x.shape)

(719, 15, 10, 2)


In [7]:
# 80/20 split
x_train, x_test, y_train, y_test = train_test_split(x, y, stratify=y, test_size=0.20, random_state=0)

In [8]:
# Check the shape of each set
print(x_train.shape)
print(x_test.shape)

(575, 15, 10, 2)
(144, 15, 10, 2)


In [9]:
# Check y balance
pd.DataFrame({
    "y": ["FB", "OFF"],
    "train": list(pd.DataFrame(y_train).value_counts()), 
    "test": list(pd.DataFrame(y_test).value_counts())})

,y,train,test
0,FB,373,93
1,OFF,202,51


# Functions

In [10]:
def split_x_by_keypoint(x):
  '''
  Convert shape of (n, 10, 15, 2, 1) to 10X (n, 15, 2, 1)
  '''

  x1, x2, x3, x4, x5, x6, x7, x8, x9, x10 = [], [], [], [], [], [], [], [], [], []

  for observation in x:
    x1.append(observation[0])
    x2.append(observation[1])
    x3.append(observation[2])
    x4.append(observation[3])
    x5.append(observation[4])
    x6.append(observation[5])
    x7.append(observation[6])
    x8.append(observation[7])
    x9.append(observation[8])
    x10.append(observation[9])

  return [
      np.array(x1), np.array(x2), np.array(x3), np.array(x4), np.array(x5), 
      np.array(x6), np.array(x7), np.array(x8), np.array(x9), np.array(x10)]
  

def evaluate_predictions(y_true, y_test_score, y_test_pred):

  # Extract metrics
  test_accuracy = round(accuracy_score(y_true, y_test_pred), 3)
  test_auc = round(roc_auc_score(y_true, y_test_score), 3)
  test_auprc = round(average_precision_score(y_true, y_test_score), 3)
  test_mcc = round(matthews_corrcoef(y_true, y_test_pred), 3)
  test_f1 = round(f1_score(y_true, y_test_pred, average='weighted'), 3)

  # Print metrics
  print(f"Accuracy: {test_accuracy}")
  print(f"AUC: {test_auc}")
  print(f"AUPRC: {test_auprc}")
  print(f"MCC: {test_mcc}") 
  print(f"F1 (Weighted): {test_f1}\n")

  return test_accuracy, test_auc, test_auprc, test_mcc, test_f1
  

# Models

In [11]:
def CNN_tc_collapse(x_shape, dropout, l1, l2, learning_rate):
  
  # Model architecture
  model = Sequential()
  model.add(Conv2D(8, (3,1), input_shape=x_shape))
  model.add(BatchNormalization())
  model.add(Activation('relu'))
  model.add(Conv2D(16, (3,1)))
  model.add(BatchNormalization())
  model.add(Activation('relu'))
  model.add(Conv2D(32, (3,1)))
  model.add(BatchNormalization())
  model.add(Activation('relu'))
  model.add(Conv2D(64, (3,1)))
  model.add(BatchNormalization())
  model.add(Activation('relu'))
  model.add(Flatten())
  model.add(Dense(256, activation='relu'))
  model.add(Dropout(dropout))
  model.add(Dense(128, activation='relu'))
  model.add(Dropout(dropout))
  model.add(Dense(64, activation='relu', kernel_regularizer=l1_l2(l1=l1, l2=l2)))
  model.add(Dense(32, activation='relu', kernel_regularizer=l1_l2(l1=l1, l2=l2)))
  model.add(Dense(16, activation='relu', kernel_regularizer=l1_l2(l1=l1, l2=l2)))
  model.add(Dense(1, activation="sigmoid"))

  # Compile model
  model.compile(
      loss='binary_crossentropy', 
      optimizer=tf.keras.optimizers.Adam(learning_rate),
      metrics=['accuracy', 'AUC'])
  
  return model


def CNN(x_shape, kernel_size, dropout, l1, l2, learning_rate):
  
  # Model architecture
  model = Sequential()
  model.add(Conv3D(8, kernel_size, input_shape=x_shape))
  model.add(BatchNormalization())
  model.add(Activation('relu'))
  model.add(Conv3D(16, kernel_size))
  model.add(BatchNormalization())
  model.add(Activation('relu'))
  model.add(Conv3D(32, kernel_size))
  model.add(BatchNormalization())
  model.add(Activation('relu'))
  model.add(Conv3D(64, kernel_size))
  model.add(BatchNormalization())
  model.add(Activation('relu'))
  model.add(Flatten())
  model.add(Dense(256, activation='relu'))
  model.add(Dropout(dropout))
  model.add(Dense(128, activation='relu'))
  model.add(Dropout(dropout))
  model.add(Dense(64, activation='relu', kernel_regularizer=l1_l2(l1=l1, l2=l2)))
  model.add(Dense(32, activation='relu', kernel_regularizer=l1_l2(l1=l1, l2=l2)))
  model.add(Dense(16, activation='relu', kernel_regularizer=l1_l2(l1=l1, l2=l2)))
  model.add(Dense(1, activation="sigmoid"))

  # Compile model
  model.compile(
      loss='binary_crossentropy', 
      optimizer=tf.keras.optimizers.Adam(learning_rate),
      metrics=['accuracy', 'AUC'])
  
  return model


def CNN_LSTM(x_shape, kernel_size, dropout, l1, l2, learning_rate):
  
  # Model architecture
  ## CNN
  model = Sequential()
  model.add(Conv3D(8, kernel_size, input_shape=x_shape))
  model.add(BatchNormalization())
  model.add(Activation('relu'))
  model.add(Conv3D(16, kernel_size))
  model.add(BatchNormalization())
  model.add(Activation('relu'))
  model.add(Conv3D(32, kernel_size))
  model.add(BatchNormalization())
  model.add(Activation('relu'))
  model.add(Conv3D(64, kernel_size))
  model.add(BatchNormalization())
  model.add(Activation('relu'))
  ## LSTM
  model.add(TimeDistributed(Flatten()))
  model.add(LSTM(256, return_sequences=False))
  ## Dense Layers
  model.add(Dense(256, activation='relu'))
  model.add(Dropout(dropout))
  model.add(Dense(128, activation='relu'))
  model.add(Dropout(dropout))
  model.add(Dense(64, activation='relu', kernel_regularizer=l1_l2(l1=l1, l2=l2)))
  model.add(Dense(32, activation='relu', kernel_regularizer=l1_l2(l1=l1, l2=l2)))
  model.add(Dense(16, activation='relu', kernel_regularizer=l1_l2(l1=l1, l2=l2)))
  model.add(Dense(1, activation="sigmoid"))

  # Compile model
  model.compile(
    loss='binary_crossentropy', 
    optimizer=tf.keras.optimizers.Adam(learning_rate),
    metrics=['accuracy', 'AUC'])
  
  return model


def CNN_branched(x_shape, dropout, l1, l2, learning_rate):

  # Architecture of each branch
  def branch(input):
    conv1 = Conv2D(8, (3,1), activation='relu', padding='same')(input)
    batchnorm1 = BatchNormalization()(conv1)
    conv2 = Conv2D(16, (3,1), activation='relu', padding='same')(batchnorm1)
    batchnorm2 = BatchNormalization()(conv2)
    conv3 = Conv2D(32, (3,1), activation='relu', padding='same')(batchnorm2)
    batchnorm3 = BatchNormalization()(conv3)
    conv4 = Conv2D(64, (3,1), activation='relu', padding='same')(batchnorm3)
    batchnorm4 = BatchNormalization()(conv4)
    gap = GlobalAveragePooling2D()(batchnorm4)
    flattened_layer = Flatten()(gap)
    return flattened_layer

  # Define the input layers for each input set
  input1 = Input(shape=x_shape)
  input2 = Input(shape=x_shape)
  input3 = Input(shape=x_shape)
  input4 = Input(shape=x_shape)
  input5 = Input(shape=x_shape)
  input6 = Input(shape=x_shape)
  input7 = Input(shape=x_shape)
  input8 = Input(shape=x_shape)
  input9 = Input(shape=x_shape)
  input10 = Input(shape=x_shape)

  # Define the CNN branches for each input set
  cnn1_branch = branch(input1)
  cnn2_branch = branch(input2)
  cnn3_branch = branch(input3)
  cnn4_branch = branch(input4)
  cnn5_branch = branch(input5)
  cnn6_branch = branch(input6)
  cnn7_branch = branch(input7)
  cnn8_branch = branch(input8)
  cnn9_branch = branch(input9)
  cnn10_branch = branch(input10)

  # Concatenate the output from each CNN branch
  concatenated_fc_input = concatenate([
      cnn1_branch, cnn2_branch, cnn3_branch, cnn4_branch, cnn5_branch,
      cnn6_branch, cnn7_branch, cnn8_branch, cnn9_branch, cnn10_branch])

  # Define the fully connected layers to process the concatenated features
  fc1 = Dense(256, activation='relu')(concatenated_fc_input)
  dropout1 = Dropout(dropout)(fc1)
  fc2 = Dense(128, activation='relu')(dropout1)
  dropout2 = Dropout(dropout)(fc2)
  fc3 = Dense(64, activation='relu', kernel_regularizer=l1_l2(l1=l1, l2=l2))(dropout2)
  fc4 = Dense(32, activation='relu', kernel_regularizer=l1_l2(l1=l1, l2=l2))(fc3)
  fc5 = Dense(16, activation='relu', kernel_regularizer=l1_l2(l1=l1, l2=l2))(fc4)
  output = Dense(1, activation='sigmoid')(fc5)  # Example output layer with 10 classes

  # Create the model by specifying the inputs and outputs
  model = Model(
      inputs=[input1, input2, input3, input4, input5, input6, input7, input8, input9, input10], 
      outputs=output)

  # Compile model
  model.compile(
      loss='binary_crossentropy', 
      optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
      metrics=['accuracy', 'AUC'])
  
  return model



# 5-Fold CV

In [14]:
# Training parameters
# Epochs and batch size
epochs = 200
batch_size = 16

# Model parameters
l1 = 1e-2
l2 = 1e-4
dropout = 0.3

# Calculate class weights for loss
classWeights = compute_class_weight(
    class_weight='balanced', classes=np.unique(y_train), y=y_train)
classWeights = {0: classWeights[0], 1: classWeights[1]}

# Early stopping call back to be called during training
early_stopping = EarlyStopping(monitor='val_loss', patience=50, verbose=0, mode='min')
reduce_lr_plateau = ReduceLROnPlateau(monitor='val_loss', factor=0.005, patience=10, verbose=0, mode='min')


In [15]:
# Define objects to store training history and results
cv_models = {
    "cnn": {"models": [], "histories": []},
    "cnn-lstm": {"models": [], "histories": []},
    "branched-cnn": {"models": [], "histories": []},
    "cnn-xtra": {"models": [], "histories": []}
}

cv_test_results = {
    "cnn": {"accuracy": [], "AUC": [], "AUPRC": [], "MCC": [], "F1": []},
    "cnn-lstm": {"accuracy": [], "AUC": [], "AUPRC": [], "MCC": [], "F1": []},
    "branched-cnn": {"accuracy": [], "AUC": [], "AUPRC": [], "MCC": [], "F1": []},
    "cnn-xtra": {"accuracy": [], "AUC": [], "AUPRC": [], "MCC": [], "F1": []}
}

cv_predictions = {
    "cnn": {"y_true": [], "y_score": {}, "y_pred": {}},
    "cnn-lstm": {"y_true": [], "y_score": {}, "y_pred": {}},
    "branched-cnn": {"y_true": [], "y_score": {}, "y_pred": {}},
    "cnn-xtra": {"y_true": [], "y_score": {}, "y_pred": {}}
}

cv_predictions["cnn"]["y_true"] = y_test
cv_predictions["cnn-lstm"]["y_true"] = y_test
cv_predictions["branched-cnn"]["y_true"] = y_test
cv_predictions["cnn-xtra"]["y_true"] = y_test

# Define the K-fold parameters
kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=1)

# Train, validate, and evaluate the model for each fold
for fold, (train_idx, val_idx) in enumerate(kfold.split(x_train, y_train)):
    
  print(f"Fold {fold+1}")
  
  # Get the training and validation data for this fold
  x_train_fold, y_train_fold = x_train[train_idx], y_train[train_idx]
  x_val_fold, y_val_fold = x_train[val_idx], y_train[val_idx]

  # Scale data
  scaler = StandardScaler()
  x_train_fold = scaler.fit_transform(x_train_fold.reshape(-1, x_train_fold.shape[-1])).reshape(x_train_fold.shape)
  x_val_fold = scaler.transform(x_val_fold.reshape(-1, x_val_fold.shape[-1])).reshape(x_val_fold.shape)
  x_test_fold = scaler.transform(x_test.reshape(-1, x_test.shape[-1])).reshape(x_test.shape)

  # Prepare for keras usage
  x_train_fold = x_train_fold.reshape(x_train_fold.shape[0], x_train_fold.shape[1], x_train_fold.shape[2], x_train_fold.shape[3], 1)
  x_val_fold = x_val_fold.reshape(x_val_fold.shape[0], x_val_fold.shape[1], x_val_fold.shape[2], x_val_fold.shape[3], 1)
  x_test_fold = x_test_fold.reshape(x_test_fold.shape[0], x_test_fold.shape[1], x_test_fold.shape[2], x_test_fold.shape[3], 1)

  # SHAPE: (n_batch, 15, 10, 2, 1)

  # CNN (Time/Coord Collapsed) -------------------------------------------------
  # Reshape to (n_batch, 10, 30, 1)
  # First: (n_batch, 10, 15, 2, 1)
  x_train_fold_cnnx = np.transpose(x_train_fold, (0, 2, 1, 3, 4))
  x_val_fold_cnnx = np.transpose(x_val_fold, (0, 2, 1, 3, 4))
  x_test_fold_cnnx = np.transpose(x_test_fold, (0, 2, 1, 3, 4))

  # Second: (n_batch, 10, 30, 1)
  x_train_fold_cnnx = x_train_fold_cnnx.reshape(x_train_fold_cnnx.shape[0], 10, 30, 1)
  x_val_fold_cnnx = x_val_fold_cnnx.reshape(x_val_fold_cnnx.shape[0], 10, 30, 1)
  x_test_fold_cnnx = x_test_fold_cnnx.reshape(x_test_fold_cnnx.shape[0], 10, 30, 1)

  # Set model
  cnnx_model = CNN_tc_collapse(
    x_shape=(10,30,1), 
    dropout=dropout, 
    l1=l1, l2=l2, 
    learning_rate=1e-4)

  # Train the model on the training data for this fold
  cnnx_history = cnnx_model.fit(
    x_train_fold_cnnx, y_train_fold, 
    epochs=epochs, batch_size=batch_size, 
    validation_data=(x_val_fold_cnnx, y_val_fold),
    class_weight=classWeights,
    callbacks=[early_stopping, reduce_lr_plateau],
    verbose=0)
  
  # Store model and history
  cv_models["cnn-xtra"]["models"].append(cnnx_model)
  cv_models["cnn-xtra"]["histories"].append(cnnx_history)

  # Evaluate predictions
  print("CNN (T/C Collapse)")
  y_test_score = cnnx_model.predict(x=x_test_fold_cnnx).flatten() # Make predictions
  y_test_pred = (y_test_score > 0.5).astype(np.float32)          # Get binary predictions from scores
  test_acc, test_auc, test_auprc, test_mcc, test_f1 = evaluate_predictions(
      y_true=y_test, y_test_score=y_test_score, y_test_pred=y_test_pred)
  
  # Store predictions
  cv_predictions["cnn-xtra"]["y_score"][f"{fold+1}"] = y_test_score
  cv_predictions["cnn-xtra"]["y_pred"][f"{fold+1}"] = y_test_pred

  # Store metrics
  cv_test_results["cnn-xtra"]["accuracy"].append(test_acc)
  cv_test_results["cnn-xtra"]["AUC"].append(test_auc)
  cv_test_results["cnn-xtra"]["AUPRC"].append(test_auprc)
  cv_test_results["cnn-xtra"]["MCC"].append(test_mcc)
  cv_test_results["cnn-xtra"]["F1"].append(test_f1)

  # Clear keras session
  K.clear_session()


  # CNN ------------------------------------------------------------------------
  # Set model
  cnn_model = CNN(
    x_shape=(15,10,2,1), 
    kernel_size=(3,3,1), 
    dropout=dropout, 
    l1=l1, l2=l2, 
    learning_rate=1e-4)

  # Train the model on the training data for this fold
  cnn_history = cnn_model.fit(
    x_train_fold, y_train_fold, 
    epochs=epochs, batch_size=batch_size, 
    validation_data=(x_val_fold, y_val_fold),
    class_weight=classWeights,
    callbacks=[early_stopping, reduce_lr_plateau],
    verbose=0)
  
  # Store model and history
  cv_models["cnn"]["models"].append(cnn_model)
  cv_models["cnn"]["histories"].append(cnn_history)

  # Evaluate predictions
  print("CNN")
  y_test_score = cnn_model.predict(x=x_test_fold).flatten() # Make predictions
  y_test_pred = (y_test_score > 0.5).astype(np.float32)     # Get binary predictions from scores
  test_acc, test_auc, test_auprc, test_mcc, test_f1 = evaluate_predictions(
      y_true=y_test, y_test_score=y_test_score, y_test_pred=y_test_pred)
  
  # Store predictions
  cv_predictions["cnn"]["y_score"][f"{fold+1}"] = y_test_score
  cv_predictions["cnn"]["y_pred"][f"{fold+1}"] = y_test_pred

  # Store metrics
  cv_test_results["cnn"]["accuracy"].append(test_acc)
  cv_test_results["cnn"]["AUC"].append(test_auc)
  cv_test_results["cnn"]["AUPRC"].append(test_auprc)
  cv_test_results["cnn"]["MCC"].append(test_mcc)
  cv_test_results["cnn"]["F1"].append(test_f1)

  # Clear keras session
  K.clear_session()


  # CNN-LSTM -------------------------------------------------------------------
  # Set model
  cnn_lstm_model = CNN_LSTM(
    x_shape=(15,10,2,1), 
    kernel_size=(3,3,1),
    dropout=dropout, 
    l1=l1, l2=l2, 
    learning_rate=1e-4)

  # Train the model on the training data for this fold
  cnn_lstm_history = cnn_lstm_model.fit(
    x_train_fold, y_train_fold, 
    epochs=epochs, batch_size=batch_size, 
    validation_data=(x_val_fold, y_val_fold),
    class_weight=classWeights,
    callbacks=[early_stopping, reduce_lr_plateau],
    verbose=0)
  
  # Store model and history
  cv_models["cnn-lstm"]["models"].append(cnn_lstm_model)
  cv_models["cnn-lstm"]["histories"].append(cnn_lstm_history)

  # Evaluate predictions
  print("CNN-LSTM")
  y_test_score = cnn_lstm_model.predict(x=x_test_fold).flatten() # Make predictions
  y_test_pred = (y_test_score > 0.5).astype(np.float32)          # Get binary predictions from scores
  test_acc, test_auc, test_auprc, test_mcc, test_f1 = evaluate_predictions(
      y_true=y_test, y_test_score=y_test_score, y_test_pred=y_test_pred)
  
  # Store predictions
  cv_predictions["cnn-lstm"]["y_score"][f"{fold+1}"] = y_test_score
  cv_predictions["cnn-lstm"]["y_pred"][f"{fold+1}"] = y_test_pred

  # Store metrics
  cv_test_results["cnn-lstm"]["accuracy"].append(test_acc)
  cv_test_results["cnn-lstm"]["AUC"].append(test_auc)
  cv_test_results["cnn-lstm"]["AUPRC"].append(test_auprc)
  cv_test_results["cnn-lstm"]["MCC"].append(test_mcc)
  cv_test_results["cnn-lstm"]["F1"].append(test_f1)

  # Clear keras session
  K.clear_session()


  # Branched model data prep ---------------------------------------------------
  # Reshape data to (n, 10, 15, 2, 1)
  x_train_fold_branched = np.transpose(x_train_fold, (0, 2, 1, 3, 4))
  x_val_fold_branched = np.transpose(x_val_fold, (0, 2, 1, 3, 4))
  x_test_fold_branched = np.transpose(x_test_fold, (0, 2, 1, 3, 4))

  # Reshape data to 10 * (n, 15, 2, 1)
  x_train_fold_branched = split_x_by_keypoint(x_train_fold_branched)
  x_val_fold_branched = split_x_by_keypoint(x_val_fold_branched)
  x_test_fold_branched = split_x_by_keypoint(x_test_fold_branched)


  # Branched-CNN ---------------------------------------------------------------
  # Set model
  branched_cnn_model = CNN_branched(
    x_shape=(15,2,1), 
    dropout=dropout, 
    l1=l1, l2=l2, 
    learning_rate=1e-4)

  # Train the model on the training data for this fold
  branched_cnn_history = branched_cnn_model.fit(
    x_train_fold_branched, y_train_fold, 
    epochs=epochs, batch_size=batch_size, 
    validation_data=(x_val_fold_branched, y_val_fold),
    class_weight=classWeights,
    callbacks=[early_stopping, reduce_lr_plateau],
    verbose=0)
  
  # Store model and history
  cv_models["branched-cnn"]["models"].append(branched_cnn_model)
  cv_models["branched-cnn"]["histories"].append(branched_cnn_history)

  # Evaluate predictions
  print("Branched-CNN")
  y_test_score = branched_cnn_model.predict(x=x_test_fold_branched).flatten() # Make predictions
  y_test_pred = (y_test_score > 0.5).astype(np.float32)                       # Get binary predictions from scores
  test_acc, test_auc, test_auprc, test_mcc, test_f1 = evaluate_predictions(
      y_true=y_test, y_test_score=y_test_score, y_test_pred=y_test_pred)
  
  # Store predictions
  cv_predictions["branched-cnn"]["y_score"][f"{fold+1}"] = y_test_score
  cv_predictions["branched-cnn"]["y_pred"][f"{fold+1}"] = y_test_pred

  # Store metrics
  cv_test_results["branched-cnn"]["accuracy"].append(test_acc)
  cv_test_results["branched-cnn"]["AUC"].append(test_auc)
  cv_test_results["branched-cnn"]["AUPRC"].append(test_auprc)
  cv_test_results["branched-cnn"]["MCC"].append(test_mcc)
  cv_test_results["branched-cnn"]["F1"].append(test_f1)

  # Clear keras session
  K.clear_session()



Fold 1
CNN (T/C Collapse)
5/5 [==============================] - 0s 2ms/step
Accuracy: 0.882
AUC: 0.917
AUPRC: 0.854
MCC: 0.738
F1 (Weighted): 0.88

CNN
5/5 [==============================] - 0s 2ms/step
Accuracy: 0.847
AUC: 0.907
AUPRC: 0.816
MCC: 0.666
F1 (Weighted): 0.847

CNN-LSTM
5/5 [==============================] - 0s 3ms/step
Accuracy: 0.854
AUC: 0.907
AUPRC: 0.816
MCC: 0.675
F1 (Weighted): 0.85

Branched-CNN
5/5 [==============================] - 1s 6ms/step
Accuracy: 0.799
AUC: 0.867
AUPRC: 0.788
MCC: 0.551
F1 (Weighted): 0.796

Fold 2
CNN (T/C Collapse)
5/5 [==============================] - 0s 3ms/step
Accuracy: 0.882
AUC: 0.919
AUPRC: 0.834
MCC: 0.738
F1 (Weighted): 0.88

CNN
5/5 [==============================] - 0s 3ms/step
Accuracy: 0.819
AUC: 0.89
AUPRC: 0.808
MCC: 0.602
F1 (Weighted): 0.819

CNN-LSTM
5/5 [==============================] - 0s 3ms/step
Accuracy: 0.819
AUC: 0.868
AUPRC: 0.825
MCC: 0.595
F1 (Weighted): 0.814

Branched-CNN
5/5 [===========================

In [16]:
pd.DataFrame({
  "cnn": round(pd.DataFrame(cv_test_results["cnn"]).mean(), 3),
  "cnn-ltsm": round(pd.DataFrame(cv_test_results["cnn-lstm"]).mean(), 3),
  "branched-cnn": round(pd.DataFrame(cv_test_results["branched-cnn"]).mean(), 3),
  "cnn-xtra": round(pd.DataFrame(cv_test_results["cnn-xtra"]).mean(), 3),
})

,cnn,cnn-ltsm,branched-cnn,cnn-xtra
accuracy,0.822,0.828,0.822,0.871
AUC,0.896,0.889,0.877,0.919
AUPRC,0.809,0.810,0.774,0.839
MCC,0.605,0.614,0.606,0.714
F1,0.820,0.823,0.819,0.869


In [17]:
history_file = os.path.join(root_path, "modeling/saved-modeling-results/kfold-model-history.glasnow.pkl")
with open(history_file, 'wb') as pkl_file:
  pickle.dump(cv_models, pkl_file)

In [18]:
results_file = os.path.join(root_path, "modeling/saved-modeling-results/kfold-model-results.glasnow.pkl")
with open(results_file, 'wb') as pkl_file:
  pickle.dump(cv_test_results, pkl_file)

In [19]:
predictions_file = os.path.join(root_path, "modeling/saved-modeling-results/kfold-model-predictions.glasnow.pkl")
with open(predictions_file, 'wb') as pkl_file:
  pickle.dump(cv_predictions, pkl_file)